In [15]:
import os
import sys
import math
import json
path='F:/TREC2015/Analysis'
os.chdir(path)
path = 'F:/TREC2015\Analysis\Nov-2017-result\jm result'
profile=['MB226','MB254','MB265','MB267','MB362','MB371','MB377','MB391','MB392','MB409','MB419']

In [14]:
ip=[]
with open('tweet_id-final18.txt','r') as fin:
    line= fin.readlines()
    for l in line:
        l=l.strip().split(' ')
        #tpn=l[0]
        tpid=l[1]
        ip.append(tpid)      
print len(ip)

51


In [16]:
import pandas as pd
import numpy as np


qrels = pd.read_csv('qrels.txt', sep=' ',header=None)
qrels.drop(qrels[[1]],axis=1,inplace=True)

#runfile.head()
epoch = pd.read_csv('tweet2dayepoch.txt', sep=' ',header=None)
#runfile=pd.read_csv('jmsmoothing20_13.txt',sep=' ',header=None)
runfile=pd.read_csv('Nov-2017-result\jm result\jm2015\JM_20.0_14.0_+4+3final_doc_l_grid_ndcg1_again_list.txt',sep=' ',header=None)
cluster = pd.read_csv('cluster.txt',sep=' ',header=None)
runfile.drop(runfile[[6]],axis=1,inplace=True)
runfile.drop(runfile[[2]],axis=1,inplace=True)


qrels.columns=['profile','tweet_id','relevance']
runfile.columns=['date','profile','tweet_id','rank','score']
runfile.head()
qrels.head()


,profile,tweet_id,relevance
0,226,622915540244369408,0
1,226,622919214433570816,0
2,226,622919248004775937,0
3,226,622919608727552000,0
4,226,622919860377419776,0


In [3]:

os.chdir(path1)
#ip=['RTS47']
for i,p in enumerate(ip):
    qn=qrels[(qrels['relevance'] > 0) & (qrels['profile'] == p[3:])]
    #qn.count()


    ij=qn.merge(epoch,how='inner',on=['tweet_id'])
    #ij.head()
    ijc= pd.merge(ij,cluster,how='inner',on=['tweet_id','profile'])
    ijc=ijc.sort_values(by=['date','time'],ascending=[True,True])
    
    os.chdir(path1)
#os.mkdir(p)
    path2=path1 +"/" + p
    os.chdir(path2)

    fn = p + '_qrel.txt'
    #ijc=ijc.sort_values(ijc.columns[3],ascending=[True])
    ijc.to_csv(fn,index=False,sep=' ',header=None)
    #textstring=  p +'text.txt'
    #tqrel.to_csv(textstring,index=False,sep= ' ')
    
    fnstr= p + '_qrel.txt'
    fnstr1 = p + '.txt'
    gq = pd.read_csv(fnstr, sep=' ')
    #gq.columns
#gq.info()
    pt=pd.read_csv(fnstr1,sep=',',header=None)
    pt.columns=['date','profile','qo','tweet_id','rank','score','text']
    
    pt.drop(pt[[2]],axis=1,inplace=True)
    pt1=pt.drop_duplicates(subset='tweet_id')
    pt1.reset_index(drop=True)
    mt=ijc.merge(pt1,how='inner',on=['tweet_id','profile','date'])
#mt.info()
#mt.head()
#mt=mt.sort_values(mt.columns[3],mt.columns[5],ascending=[True,True])
    mt=mt.sort_values(by=['date','rank'],ascending=True)
    mt.to_csv(p +'_qrelinrank.txt',index=False,sep=' ',header=None)
    pt11=pt1['tweet_id'].tolist()
    gq1=ijc['tweet_id'].tolist()
    
    ngq=list(set(gq1)-set(pt11))
    if len(ngq) > 0 :
        ngq_df = pd.DataFrame(ngq)
        ngq_df.columns=['tweet_id']
        qrelmissRank_df= pd.merge(ngq_df,ijc,how='inner',on=['tweet_id'])
        qrelmissRank_df=qrelmissRank_df.sort_values(by=['date'],ascending=[True])
#qrelmissRank_df= pd.merge(qrelmissRank_df,epoch,how='inner',on=['tweet_id'])
        #nqrelmissRank_df= pd.merge(ngq_df,ijc,how='inner',on=['tweet_id'])
        qrelmissRank_df.to_csv(p + '_qrelmissrank.txt',index=False,sep= ' ',header=None)
    #path_rn='C:/Users/lenovo/Google Drive/PHD share folder/paper/trec experiment/rishabh/11417'

    #os.chdir(path_rn)
    os.chdir(path1)
    run_df=pd.read_csv("C:/Users/lenovo/Google Drive/PHD share folder/trec-rts-2017/runfile/JM_20.0_14.0_+4+3final_doc_l_grid_ndcg1_again_list.txt",sep=' ',header=None)
#run_df.drop(run_df[[7]],axis=1,inplace=True)
    run_df.drop(run_df[[6]],axis=1,inplace=True)

    run_df.columns=['date','profile','q0','tweet_id','rank','score']
    run_df.drop(run_df[[2]],axis=1,inplace=True)


    run_df=run_df[(run_df['profile'] == p)]
    #run_df.info()
    run_merge_df =pd.merge(mt,run_df,how='inner',on=['tweet_id','profile','date'])
    
    os.chdir(path1)
    os.chdir(p)
    run_merge_df=run_merge_df.sort_values(by=['date','rank_y'],ascending=[True,True])
    run_merge_df.to_csv(p+"_System_summary.txt",index=False,sep=' ')
    
    Qrel_notinsumm_df =  pd.DataFrame(list(set(mt['tweet_id'].tolist()) - set(run_df['tweet_id'].tolist())))
    if(Qrel_notinsumm_df.empty==False):
        Qrel_notinsumm_df.columns=["tweet_id"]
        Qrel_notinsumm_df = pd.merge(Qrel_notinsumm_df,mt,how='inner',on=['tweet_id'])
        Qrel_notinsumm_df.info()
#run_merge_df.info()
        Qrel_notinsumm_df=Qrel_notinsumm_df.sort_values(by=['date','rank'],ascending=[True,True])
        Qrel_notinsumm_df.to_csv(p +'System_notincluded.txt',index=False,sep=' ')
    
    #if not (wt_summ_df.empty):
    wt_summ_df =  pd.DataFrame(list(set(run_df['tweet_id'].tolist()) - set(mt['tweet_id'].tolist())))
    #wt_summ_df.info()
    if(wt_summ_df.empty==False):
        wt_summ_df.columns=["tweet_id"]
        wt_summ_df = pd.merge(wt_summ_df,pt1,how='inner',on='tweet_id')
    #wt_summ_df.head()
#wt_summ_df.info()
        wt_summ_df=wt_summ_df.sort_values(by=['date','rank'],ascending=[True,True])
        wt_summ_df.to_csv(p + "NonRelevantTweetinsummary.txt",index=False,sep= ' ')
    os.chdir(path1)
    if(run_merge_df.empty==False):
        a=run_merge_df['cluster'].nunique()
    else:
        a=0
    b=len(run_merge_df)

    cls = cluster[(cluster['profile'] == p)]
    if(cls.empty==False):
        c= cls['cluster'].nunique()
    else:
        c=0
    if(wt_summ_df.empty==False):
        nrt = len(wt_summ_df)
    else:
        nrt=0
    rt = b-a
    statstr = p + " " + str(c) + " " + str(a) + " " + str (rt) + " " + str(nrt)+"\n"
    #os.chdir(os.pardir)
    fstat = open('stats.txt',"a+")
    fstat.write(statstr)
    fstat.close()

94066 91060
<class 'pandas.core.frame.DataFrame'>
Int64Index: 118 entries, 66460 to 93080
Data columns (total 3 columns):
profile      118 non-null int64
tweet_id     118 non-null int64
relevance    118 non-null int64
dtypes: int64(3)
memory usage: 3.7 KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1292 entries, 66430 to 93110
Data columns (total 3 columns):
profile      1292 non-null int64
tweet_id     1292 non-null int64
relevance    1292 non-null int64
dtypes: int64(3)
memory usage: 40.4 KB


In [4]:
print "total qrels for 1:",qrels[qrels['relevance'] == 3 ].count()
print "total qrels for 2:",qrels[qrels['relevance'] == 4 ].count()
print "total qrels for 1:",qrels[qrels['relevance'] == 1 ].count()
print "total qrels for 2:",qrels[qrels['relevance'] == 2 ].count()

total qrels for 1: profile      391
tweet_id     391
relevance    391
dtype: int64
total qrels for 2: profile      1651
tweet_id     1651
relevance    1651
dtype: int64
total qrels for 1: profile      3032
tweet_id     3032
relevance    3032
dtype: int64
total qrels for 2: profile      3159
tweet_id     3159
relevance    3159
dtype: int64


In [5]:
runq = pd.merge(runfile,qrels,how='inner',on=['tweet_id'])


In [6]:
runq.head()
runq.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13 entries, 0 to 12
Data columns (total 7 columns):
date         13 non-null int64
profile_x    13 non-null object
tweet_id     13 non-null int64
rank         13 non-null int64
score        13 non-null float64
profile_y    13 non-null int64
relevance    13 non-null int64
dtypes: float64(1), int64(5), object(1)
memory usage: 832.0+ bytes


In [7]:
runfile.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7017 entries, 0 to 7016
Data columns (total 5 columns):
date        7017 non-null int64
profile     7017 non-null object
tweet_id    7017 non-null int64
rank        7017 non-null int64
score       7017 non-null float64
dtypes: float64(1), int64(3), object(1)
memory usage: 274.2+ KB


In [8]:
runq.head()

,date,profile_x,tweet_id,rank,score,profile_y,relevance
0,20150724,MB276,624587322709815296,4,0.784318,326,1
1,20150724,MB371,624657115940937728,29,0.417220,326,0
2,20150721,MB440,623426515506720769,14,0.479027,236,0
3,20150721,MB436,623498154252574720,11,0.575879,284,1
4,20150721,MB258,623498154252574720,20,0.492924,284,1


In [9]:
epoch.columns=['tweet_id','date','num']
epoch=epoch.drop_duplicates(subset=['tweet_id','date'])
qe=pd.merge(qrels,epoch,how='inner',on=['tweet_id'])
epoch.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 71366 entries, 0 to 71365
Data columns (total 3 columns):
tweet_id    71366 non-null int64
date        71366 non-null int64
num         71366 non-null int64
dtypes: int64(3)
memory usage: 2.2 MB


In [35]:
qe.to_csv('qrelwithdate.txt',index=False)

In [6]:
qrels['tweet_id'].nunique()

91060

In [15]:
qrelsid=pd.DataFrame(qrels['tweet_id'])
type(qrelsid)
qrelsid=qrelsid.drop_duplicates(subset=['tweet_id'],keep='first')
qrelsid.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 91060 entries, 0 to 94065
Data columns (total 1 columns):
tweet_id    91060 non-null int64
dtypes: int64(1)
memory usage: 1.4 MB


In [32]:
tweet_df=pd.merge(qrelsid,epoch,how='inner',on=['tweet_id'])
tweet_df.info()
data=[20150720,20150721,20150722,20150723,20150724,20150725,20150726,20150727,20150728,20150729]
for i in data :
    temp =tweet_df[tweet_df['date']==i]
    temp.info()
    temp_df = pd.DataFrame(temp['tweet_id'])
    temp_df.to_csv(str(i)+'.txt',header=None,index=False)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 71366 entries, 0 to 71365
Data columns (total 3 columns):
tweet_id    71366 non-null int64
date        71366 non-null int64
num         71366 non-null int64
dtypes: int64(3)
memory usage: 2.2 MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 6346 entries, 1 to 69987
Data columns (total 3 columns):
tweet_id    6346 non-null int64
date        6346 non-null int64
num         6346 non-null int64
dtypes: int64(3)
memory usage: 198.3 KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 7576 entries, 144 to 70149
Data columns (total 3 columns):
tweet_id    7576 non-null int64
date        7576 non-null int64
num         7576 non-null int64
dtypes: int64(3)
memory usage: 236.8 KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 7255 entries, 301 to 70304
Data columns (total 3 columns):
tweet_id    7255 non-null int64
date        7255 non-null int64
num         7255 non-null int64
dtypes: int64(3)
memory usage: 226.7 KB
<class 'pandas.core.fr

In [23]:
qrelnot = pd.DataFrame(list(set(qrelsid['tweet_id'].tolist()) - set(epoch['tweet_id'].tolist())))
qrelnot.info()
qrelnot.to_csv('2030.txt',header=None,index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19694 entries, 0 to 19693
Data columns (total 1 columns):
0    19694 non-null int64
dtypes: int64(1)
memory usage: 153.9 KB


In [30]:

temp.head()

,tweet_id,date,num
1405,626180461954531328,20150729,1438128030
1406,626180646524854272,20150729,1438128074
1407,626180801726685185,20150729,1438128111
1408,626184085841518592,20150729,1438128894
1409,626184165529096192,20150729,1438128913


In [13]:
p

419